In [ ]:

import pandas as pd
import numpy as np
import pickle
import os
import math
import xlsxwriter
from adjustText import adjust_text
import cobra
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import FancyArrowPatch
from itertools import permutations, product, combinations
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import scipy
from scipy import stats
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
from cobra.io import read_sbml_model, write_sbml_model, save_matlab_model, load_matlab_model
from cobra.flux_analysis import flux_variability_analysis
from time import time



In [ ]:
os.environ['GUROBI_HOME'] = "/depot/pbaloni/data/Lab_members/Boyu_Jiang/Software/gurobi_license"
os.environ['GRB_LICENSE_FILE'] = "/depot/pbaloni/data/Lab_members/Boyu_Jiang/Software/gurobi_license/gurobi.lic"

## UC sutdy 

#### Run FVA with context-specific reconstructions of healthy samples in the UC study

In [ ]:
# UC healthy

files_path = os.listdir('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Broad_study/FAV_per_sample/model_imat_samples/Healthy_reconstructions/')
folder = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Broad_study/FAV_per_sample/model_imat_samples/Healthy_reconstructions/'
for i in files_path:
    model_path = folder + i
    print(model_path)
    model = cobra.io.load_matlab_model(model_path)
    model.objective = ['ACSm', 'FACOAL40im', 'BIOMASS_maintenance']
    model.solver = 'cplex'
    print(model.objective.expression)
    fva_output = flux_variability_analysis(model)
    save_path = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Broad_study/FAV_per_sample/FVA_outputs/'
    file_name = i.split('.mat')[0]
    save_path = save_path + file_name + '.csv'
    fva_output.to_csv(save_path)
    print('------------------------------')

In [ ]:
# UC inflammatory
# Run fva with context-specific reconstructions of inflammatory samples in the UC study
files_path = os.listdir('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Broad_study/FAV_per_sample/model_imat_samples/Inflamed_reconstructions/')
folder = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Broad_study/FAV_per_sample/model_imat_samples/Inflamed_reconstructions/'
for i in files_path:
    model_path = folder + i
    print(model_path)
    model = cobra.io.load_matlab_model(model_path)
    model.objective = ['ACSm', 'FACOAL40im', 'BIOMASS_maintenance']
    model.solver = 'cplex'
    print(model.objective.expression)
    fva_output = flux_variability_analysis(model)
    save_path = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Broad_study/FAV_per_sample/FVA_outputs/'
    file_name = i.split('.mat')[0]
    save_path = save_path + file_name + '.csv'
    fva_output.to_csv(save_path)
    print('------------------------------')

In [ ]:
# Merge maximum flux of the FVA outputs above
all_rxn = []
length = 1

for filename in os.listdir('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Broad_study/FAV_per_sample/FAV_output'):
    print(filename)    
    filePath = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Broad_study/FAV_per_sample/FAV_output/' + filename
    fva = pd.read_csv(filePath)
    rxn = fva.iloc[:,0].to_list()
    length += len(rxn)
    all_rxn += rxn
print(length)
print(len(all_rxn))
reaction_deduplicated = list(set(all_rxn))

print("merge fVA values--------------------")
# merge all fva maximum value
workbook =xlsxwriter.Workbook('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Broad_study/FAV_per_sample/Results_analysis/FVA_mergedAll.xlsx', {"nan_inf_to_errors": True})
sheet = workbook.add_worksheet()
for i in range(len(reaction_deduplicated)):
    reaction = reaction_deduplicated[i]
    sheet.write(i+1, 0, reaction)

index = 1
file_list = os.listdir('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Broad_study/FAV_per_sample/FAV_output/')
for filename in sorted(file_list):
    print(filename)
    sample_name = filename[:-8]
    filePath = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Broad_study/FAV_per_sample/FAV_output/' + filename
    sheet.write(0,index, sample_name)
    

    fva = pd.read_csv(filePath, index_col=0)
    fva_rxn = fva.index.to_list()
    for i in range(len(reaction_deduplicated)):
        reaction = reaction_deduplicated[i]
        if reaction in fva_rxn:
            value = fva.loc[reaction, 'maximum']
            sheet.write(i+1, index, value)
        else:
            value = None
            sheet.write(i+1, index, value)
    index += 1


workbook.close()

In [ ]:
# UC T-test 
UC_data = pd.read_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Broad_study/FAV_per_sample/Results_analysis/FVA_mergedAll.xlsx', index_col=0)


p_val = []
for i in range(4817):
    rxn_id = UC_data.index[i]
    rxn_fluxes = UC_data.loc[rxn_id].tolist()
    ifla_flux = rxn_fluxes[0:3]
    ht_flux  = rxn_fluxes[3:]
    try:
        t_statistic, p_value = ttest_ind(ifla_flux, ht_flux)
        p_val.append(p_value)
    except:
        p_val.append(np.nan)

UC_data['p_value'] = p_val
UC_data.to_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Broad_study/FAV_per_sample/Results_analysis/UC_t_test.xlsx')

#### Run Flux sampling with all context-specific reconstructions in the UC study

In [ ]:
UC_folder = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Broad_study/FAV_per_sample/model_imat_samples/'

for i in ['Healthy_reconstructions', 'Inflamed_reconstructions']:
    folder =  UC_folder + i
    print(folder)
    file_names = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    print(file_names)

    for i in range(len(file_names)):
        print(i)
        filename  = file_names[i]
        print(filename)
        file_path = folder + '/' + filename
        model = load_matlab_model(file_path)
        # model.objective = ['ACSm', 'FACOAL40im', 'BIOMASS_maintenance']
        try:
            achr = ACHRSampler(model, thinning=10)
            s1 = achr.sample(1000)
            save_path = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Flux sampling/UC/'
            save_file_name = filename.split('.xml')[0]
            save_file = save_path + save_file_name + '.csv'
            s1.to_csv(save_file)
        except:
            print('cannnot obtain sampling')

In [ ]:
def pad_lists_to_same_length(data, pad_value=np.nan):
    max_length = max(len(lst) for lst in data.values())
    for key, lst in data.items():
        if len(lst) < max_length:
            data[key].extend([pad_value] * (max_length - len(lst)))
    return data

In [ ]:
# combined fluxes from each group of UC
UC_flux_files = os.listdir('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Flux sampling/UC')
UC_flux_path = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Flux sampling/UC/'

# UC_inflamed 
inflam_dict = {}
for i in UC_flux_files:
    if i.startswith('Inflame'):
        file_path = UC_flux_path + '/' + i
        fluxes_data = pd.read_csv(file_path, index_col=0)
        rxn_list = fluxes_data.columns.to_list()
        for i in rxn_list:
            rxn = i
            rxn_flux = fluxes_data[rxn].to_list()
            if rxn not in inflam_dict.keys():
                inflam_dict[rxn] = rxn_flux
            else:
                ori_flux = inflam_dict[rxn]
                combined_flux = ori_flux + rxn_flux
                inflam_dict[rxn] = combined_flux
        inflam_dict = pad_lists_to_same_length(inflam_dict)
df_inflam = pd.DataFrame(inflam_dict)
df_inflam.to_csv('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Flux sampling/Output/UC_inflam_combined.csv')




health_dict = {}
for i in UC_flux_files:
    if i.startswith('Health'):
        file_path = UC_flux_path + '/' + i
        fluxes_data = pd.read_csv(file_path, index_col=0)
        rxn_list = fluxes_data.columns.to_list()
        for i in rxn_list:
            rxn = i
            rxn_flux = fluxes_data[rxn].to_list()
            if rxn not in health_dict.keys():
                health_dict[rxn] = rxn_flux
            else:
                ori_flux = health_dict[rxn]
                combined_flux = ori_flux + rxn_flux
                health_dict[rxn] = combined_flux
        health_dict = pad_lists_to_same_length(health_dict)
df_HT = pd.DataFrame(health_dict)
df_HT.to_csv('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Flux sampling/Output/UC_HT_combined.csv')



UC_inflammation = pd.read_csv('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Flux sampling/Output/UC_inflam_combined.csv', index_col=0)
UC_health = pd.read_csv('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Flux sampling/Output/UC_HT_combined.csv', index_col=0)

workbook = xlsxwriter.Workbook('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Flux sampling/Output/UC_combinedFlux_comparsion.xlsx')
sheet1 = workbook.add_worksheet()


sheet1.write(0, 0, 'UC_inflammation_vs_health')
sheet1.write(1, 0, 'Reactions')
sheet1.write(1, 1, 'Subsystem')
sheet1.write(1, 2, 'Flux_CD')
sheet1.write(1, 3, 'Flux_HT')
sheet1.write(1, 4, 'P-value')
inflam_rxns = UC_inflammation.columns.to_list()
health_rxns = UC_health.columns.to_list()
overlapped_rxn = list(set(inflam_rxns).intersection(health_rxns))

row_index = 2
for rxn in overlapped_rxn:
    print(rxn)
    sheet1.write(row_index, 0, rxn)
    
    inflam_fluxes = UC_inflammation[rxn].to_list()
    inflam_fluxes = [x for x in inflam_fluxes if not math.isnan(x)]
    health_fluxes = UC_health[rxn].to_list()
    health_fluxes = [x for x in health_fluxes if not math.isnan(x)]
    inflam_fluxes_mean = mean(inflam_fluxes)
    health_fluxes_mean = mean(health_fluxes)
    sheet1.write(row_index, 2, inflam_fluxes_mean)
    sheet1.write(row_index, 3, health_fluxes_mean)
    t_statistic, p_value = ttest_ind(inflam_fluxes, health_fluxes)
    sheet1.write(row_index, 4, p_value)
    row_index +=1

workbook.close()

In [ ]:
# build combined flux distribution plot of UC
metabolic_tasks = pd.read_csv('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Flux sampling/input_metabolic_tasks.csv')
metabolic_tasks = metabolic_tasks['Reaction_Recon3D'].to_list()
UC_inflam_fluxes = pd.read_csv('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Flux sampling/Output/UC_inflam_combined.csv', index_col=0)
UC_health_fluxes = pd.read_csv('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Flux sampling/Output/UC_HT_combined.csv', index_col=0)


save_folder = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Flux sampling/DistributionPlots/'

for i in metabolic_tasks:
    reaction = i

    try:
        UC_inflam_data = UC_inflam_fluxes[reaction].to_list()
        UC_inflam_data = [x for x in UC_inflam_data if not math.isnan(x)]
        UC_health_data = UC_health_fluxes[reaction].to_list()
        UC_health_data = [x for x in UC_health_data if not math.isnan(x)]

        # Sample data
        data1 = UC_inflam_data
        data2 = UC_health_data

        # Create distribution plot for data1 and data2
        sns.histplot(data1, kde=True, color='red', label='Inflammatory group', alpha=0.5)
        sns.histplot(data2, kde=True, color='blue', label='Healthy group', alpha=0.5)

        plt.xlabel('Flux value (mmol/gDW/h)')
        plt.ylabel('Density')
        Plot_title = 'UC study: ' + reaction
        plt.title(Plot_title)
        plt.legend()
        save_path = save_folder + 'UC_' + reaction + '.png'
        plt.savefig(save_path,dpi=600)
        plt.show()
    except:
        continue

#### Single gene knockout simulation in UC

In [ ]:
icolon = read_sbml_model('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Draft_reconstructions/Draft_reconstruction_consensus/icolonoEpithelium.xml')
reactions_all = ['BIOMASS_maintenance', 'ACSm', 'FACOAL40im', 
                 'HACD1m',
                 'ECOAH1m',
                 '5HOXINDACTOX',
                 'ECOAH1x',
                 'HACD1x',
                 '3HKYNAKGAT',
                 '5HOXINOXDA',
                 'RE2349C',
                 'r0647',
                 'ACACT1m']

merged_df = pd.DataFrame({'reaction_id': reactions_all})
merged_df.index = merged_df['reaction_id']



files_path = os.listdir('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Broad_study/FAV_per_sample/model_imat_samples/Healthy_reconstructions/')
folder = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Broad_study/FAV_per_sample/model_imat_samples/Healthy_reconstructions/'
print(files_path)

for i in range(len(files_path)):
    print(i)
    file_name = files_path[i]
    print(file_name)
    merged_df = pd.DataFrame({'reaction_id': reactions_all})
    merged_df.index = merged_df['reaction_id']
    
    save_path = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_UC/' + file_name + '_knock.xlsx'

    
    
    model_path = folder + file_name
    model = load_matlab_model(model_path)
    model.solver = 'cplex'
    print(i)
    index = 0
    reaction = [i.id for i in model.reactions]
    reactions_all_filter = list(set(reactions_all)&set(reaction))
    for i in model.genes:
        
        if index % 100 == 0:
            print(index)
        index +=1 
        gene_id = i.id
        
        model_copy = model.copy()
        model_copy.genes.get_by_id(gene_id).knock_out()
        fva = flux_variability_analysis(model_copy, reactions_all_filter, processes = 9)
        fva = fva.drop(columns=['minimum'])
        fva.rename(columns={'maximum': gene_id}, inplace=True)
        merged_df = pd.merge(merged_df,fva, left_index=True, right_index=True, how='left')
        
    merged_df.to_excel(save_path)
    print('complete knockout:', file_name)


files_path = os.listdir('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Broad_study/FAV_per_sample/model_imat_samples/Inflamed_reconstructions/')
folder = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Broad_study/FAV_per_sample/model_imat_samples/Inflamed_reconstructions/'
print(files_path)

for i in range(len(files_path)):
    print(i)
    file_name = files_path[i]
    print(file_name)
    merged_df = pd.DataFrame({'reaction_id': reactions_all})
    merged_df.index = merged_df['reaction_id']
    
    save_path = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_UC/' + file_name + '_knock.xlsx'

    
    model_path = folder + file_name
    model = load_matlab_model(model_path)
    reaction = [i.id for i in model.reactions]
    reactions_all_filter = list(set(reactions_all)&set(reaction))
    model.solver = 'cplex'

    index = 0
    for i in model.genes:
        
        if index % 100 == 0:
            print(index)
        index +=1 
        gene_id = i.id
        
        model_copy = model.copy()
        model_copy.genes.get_by_id(gene_id).knock_out()
        fva = flux_variability_analysis(model_copy, reactions_all_filter, processes = 6)
        fva = fva.drop(columns=['minimum'])
        fva.rename(columns={'maximum': gene_id}, inplace=True)
        merged_df = pd.merge(merged_df,fva, left_index=True, right_index=True, how='left')
        
    merged_df.to_excel(save_path)
    print('knockout completed:', file_name)



In [ ]:
# process data
reactions_all = [i.id for i in icolon.genes]
merged_df = pd.DataFrame({'': reactions_all})
merged_df.index = merged_df['']
                             
merged_biomass = merged_df['']
merged_acsm = merged_df['']
merge_FACOAL40im = merged_df['']
merge_ACACT1m = merged_df['']
merge_ECOAH1m = merged_df['']
merge_5HOXINOXDA = merged_df['']
merge_5HOXINDACTOX = merged_df['']
merge_3HKYNAKGAT = merged_df['']


files = os.listdir('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_UC')
folder = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_UC/'
files.remove('processed')
files.remove('plot_dada')
files.remove('Figure')

files.sort()

for i in files:
    print(i)
    sample_id = i.split('.mat_knock.xlsx')[0]
    print(sample_id)
    file_path = folder + i
    df_ko = pd.read_excel(file_path)
    df_ko.index = df_ko['reaction_id']

    selected_rxn = ['BIOMASS_maintenance', 
                    'ACSm', 
                    'FACOAL40im', 
                    'ACACT1m', 
                    'ECOAH1m', 
                    '5HOXINOXDA', 
                    '5HOXINDACTOX', 
                    '3HKYNAKGAT']

    df_ko = df_ko.loc[selected_rxn]
    

    col1 = sample_id + '_BIOMASS_maintenance'
    col2 = sample_id + '_ACSm'
    col3 = sample_id + '_FACOAL40im'
    col4 = sample_id + '_ACACT1m'
    col5 = sample_id + '_ECOAH1m'
    col6 = sample_id + '_5HOXINOXDA'
    col7 = sample_id + '_5HOXINDACTOX'
    col8 = sample_id + '_3HKYNAKGAT'

    df_ko_t = df_ko.T
    df_ko_t.columns =[col1, col2, col3, col4, col5, col6, col7, col8]
    df_ko_t = df_ko_t.iloc[2:]


    wt_df_folder = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Broad_study/FAV_per_sample/FAV_outputs_0521_2024_lu_0/'
    wt_df_folder_path = wt_df_folder  + sample_id + '.csv'
    wt_df = pd.read_csv(wt_df_folder_path, index_col=0)
   
    wt_BIOMASS = wt_df.loc['BIOMASS_maintenance', 'maximum']
    if wt_BIOMASS == 0:
        wt_BIOMASS = 1e-8
        print('set wt_BIOMASS as 1e-8')
    else:
        print(wt_BIOMASS)

    wt_acsm = wt_df.loc['ACSm', 'maximum']
    if wt_acsm == 0:
        wt_acsm = 1e-8
        print('set wt_acsm as 1e-8')
    else:
        print(wt_acsm)
   
    wt_FACOAL40im = wt_df.loc['FACOAL40im', 'maximum']
    if wt_FACOAL40im == 0:
        wt_FACOAL40im = 1e-8
        print('set wt_FACOAL40im as 1e-8')
    else:
        print(wt_FACOAL40im)

    try:
        wt_ACACT1m = wt_df.loc['ACACT1m', 'maximum']
        if wt_ACACT1m == 0:
            wt_ACACT1m = 1e-8
            print('set wt_ACACT1m as 1e-8')
        else:
            print(wt_ACACT1m)
    except:
        wt_ACACT1m = 1e-8
        print('set wt_ACACT1m as 1e-8')

    try:
        wt_ECOAH1m = wt_df.loc['ECOAH1m', 'maximum']
        if wt_ECOAH1m == 0:
            wt_ECOAH1m = 1e-8
            print('set wt_ECOAH1m as 1e-8')
        else:
            print(wt_ECOAH1m)
    except:
        wt_ECOAH1m = 1e-8
        print('set wt_ECOAH1m as 1e-8')


    try:
        wt_5HOXINDACTOX = wt_df.loc['5HOXINDACTOX', 'maximum']
        if wt_5HOXINDACTOX == 0:
            wt_5HOXINDACTOX = 1e-8
            print('set wt_5HOXINDACTOX as 1e-8')
        else:
            print(wt_5HOXINDACTOX)
    except:
        wt_5HOXINDACTOX = 1e-8
        print('set wt_5HOXINDACTOX as 1e-8')

    
    try:
        wt_5HOXINOXDA = wt_df.loc['5HOXINOXDA', 'maximum']
        if wt_5HOXINOXDA == 0:
            wt_5HOXINOXDA = 1e-8
            print('set wt_5HOXINOXDA as 1e-8')
        else:
            print(wt_5HOXINOXDA)
    except:
        wt_5HOXINOXDA = 1e-8
        print('set wt_5HOXINOXDA as 1e-8')


    try:
        wt_3HKYNAKGAT = wt_df.loc['3HKYNAKGAT', 'maximum']
        if wt_3HKYNAKGAT == 0:
            wt_3HKYNAKGAT = 1e-8
            print('set wt_3HKYNAKGAT as 1e-8')
        else:
            print(wt_3HKYNAKGAT)
    except:
        wt_3HKYNAKGAT = 1e-8
        print('set wt_3HKYNAKGAT as 1e-8')


    

    BIOMASS = df_ko_t.iloc[:, 0:1]
    BIOMASS = (wt_BIOMASS-BIOMASS)/wt_BIOMASS
    merged_biomass = pd.merge(merged_biomass, BIOMASS, left_index=True, right_index=True, how='left')

    acsm = df_ko_t.iloc[:, 1:2]
    acsm = (wt_acsm-acsm)/wt_acsm
    merged_acsm = pd.merge(merged_acsm, acsm, left_index=True, right_index=True, how='left')

    FACOAL40im = df_ko_t.iloc[:, 2:3]
    FACOAL40im = (wt_FACOAL40im-FACOAL40im)/wt_FACOAL40im
    merge_FACOAL40im = pd.merge(merge_FACOAL40im, FACOAL40im, left_index=True, right_index=True, how='left')

    ACACT1m = df_ko_t.iloc[:, 3:4]
    ACACT1m = (wt_ACACT1m-ACACT1m)/wt_ACACT1m
    merge_ACACT1m = pd.merge(merge_ACACT1m, ACACT1m, left_index=True, right_index=True, how='left')

    ECOAH1m = df_ko_t.iloc[:, 4:5]
    ECOAH1m = (wt_ECOAH1m-ECOAH1m)/wt_ECOAH1m
    merge_ECOAH1m = pd.merge(merge_ECOAH1m, ECOAH1m, left_index=True, right_index=True, how='left')

    _5HOXINOXDA = df_ko_t.iloc[:, 5:6]
    _5HOXINOXDA = (wt_5HOXINOXDA-_5HOXINOXDA)/wt_5HOXINOXDA
    merge_5HOXINOXDA = pd.merge(merge_5HOXINOXDA, _5HOXINOXDA, left_index=True, right_index=True, how='left')

    _5HOXINDACTOX = df_ko_t.iloc[:, 6:7]
    _5HOXINDACTOX = (wt_5HOXINDACTOX-_5HOXINDACTOX)/wt_5HOXINDACTOX
    merge_5HOXINDACTOX = pd.merge(merge_5HOXINDACTOX, _5HOXINDACTOX, left_index=True, right_index=True, how='left')

    _3HKYNAKGAT = df_ko_t.iloc[:, 7:8]
    _3HKYNAKGAT = (wt_3HKYNAKGAT-_3HKYNAKGAT)/wt_3HKYNAKGAT
    merge_3HKYNAKGAT = pd.merge(merge_3HKYNAKGAT, _3HKYNAKGAT, left_index=True, right_index=True, how='left')

    

merged_biomass = merged_biomass.drop(merged_biomass.columns[0], axis=1)
merged_acsm = merged_acsm.drop(merged_acsm.columns[0], axis=1)
merge_FACOAL40im = merge_FACOAL40im.drop(merge_FACOAL40im.columns[0], axis=1)
merge_ACACT1m = merge_ACACT1m.drop(merge_ACACT1m.columns[0], axis=1)
merge_ECOAH1m = merge_ECOAH1m.drop(merge_ECOAH1m.columns[0], axis=1)
merge_5HOXINOXDA = merge_5HOXINOXDA.drop(merge_5HOXINOXDA.columns[0], axis=1)
merge_5HOXINDACTOX = merge_5HOXINDACTOX.drop(merge_5HOXINDACTOX.columns[0], axis=1)
merge_3HKYNAKGAT = merge_3HKYNAKGAT.drop(merge_3HKYNAKGAT.columns[0], axis=1)   
    



merged_biomass.to_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_UC/processed/UC_BIOMASS.xlsx')

merged_acsm.to_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_UC/processed/UC_ACSm.xlsx')

merge_FACOAL40im.to_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_UC/processed/UC_FACOAL40im.xlsx')

merge_ACACT1m.to_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_UC/processed/UC_ACACT1m.xlsx')

merge_ECOAH1m.to_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_UC/processed/UC_ECOAH1m.xlsx')

merge_5HOXINOXDA.to_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_UC/processed/UC_5HOXINOXDA.xlsx')

merge_5HOXINDACTOX.to_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_UC/processed/UC_5HOXINDACTOX.xlsx')

merge_3HKYNAKGAT.to_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_UC/processed/UC_3HKYNAKGAT.xlsx')


In [ ]:
folder = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_UC/processed/'
files = os.listdir(folder)

for i in files:
    file_path = folder + i
    print(i)
    df = pd.read_excel(file_path, index_col=0)
    df.sort_values(by=df.columns.tolist(), ascending=True, inplace=True)
    cd_mean  = df.iloc[:, :3].mean(axis=1)
    ht_mean  = df.iloc[:, 3:7].mean(axis=1)
    all_mean = df.iloc[:, 0:7].mean(axis=1)
    ratio = abs(cd_mean/ht_mean)
    try:
        ratio_log2 = ratio.apply(lambda x: math.log2(x))
    except:
        #cd_mean  = df.iloc[:, :3].mean(axis=1)
        #cd_mean = cd_mean.replace(0, 1e-8)
        #ht_mean  = df.iloc[:, 3:7].mean(axis=1)
        #all_mean = df.iloc[:, 0:7].mean(axis=1)
        #ratio = abs(cd_mean/ht_mean)
        #ratio_log2 = ratio.apply(lambda x: math.log2(x))
        continue
    ratio_log2 = ratio_log2.to_frame()
    
    # t-test
    cd = df.iloc[:, :3]
    ht = df.iloc[:, 3:7]
    t_statistic, p_values = ttest_ind(cd, ht, axis=1, nan_policy='omit')

    geneID = cd.index
    geneID = geneID.to_list()
    p_values = list(p_values)
    ttest_results = pd.DataFrame({'GeneID':geneID, 'p-valve': p_values})
    ttest_results.index = ttest_results['GeneID']
    ttest_results = ttest_results.drop('GeneID', axis=1)
    
    ratio_log2.rename(columns={ratio_log2.columns[0]: 'log2(CD/WT)'}, inplace=True)
    ratio_log2['all mean'] = all_mean
    volcano_data = pd.merge(ratio_log2, ttest_results, left_index=True, right_index=True, how='left')
    volcanoPlot_data_save = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_UC/plot_dada/' + i
    print(volcanoPlot_data_save)
    volcano_data.to_excel(volcanoPlot_data_save)

In [ ]:
# build volcano plot 
volcano_data = pd.read_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_UC/plot_dada/UC_BIOMASS.xlsx', index_col=0)

log_fold_change = volcano_data['log2(CD/WT)']
p_values = volcano_data['p-valve']
p_values = -np.log10(p_values)
all_mean = volcano_data['all mean']
gene_id = volcano_data.index.to_list()
threshold = -np.log10(0.05)

sizes = np.linspace(1, 250, len(all_mean))
colors = ['red' if fc > 0 else 'blue' for fc in log_fold_change]

plt.figure(figsize=(8, 6), dpi=900)
plt.scatter(log_fold_change, p_values, s=sizes, c=colors, alpha=0.5)

text = []
font_props = {'family': 'monospace', 'size': 8, 'weight': 'bold'}
for x, y, l in zip(log_fold_change, p_values, gene_id):
    if y > -np.log10(0.05):
        text.append(plt.text(x, y, l, size=10, ha='center', va='center', fontdict=font_props))

adjust_text(text, expand=(2, 5),arrowprops=dict(arrowstyle="->", color='k', lw=1))       
plt.xlabel('log2(Inflammatory group/Healthy group)')
plt.ylabel('-log10(p-value)')
plt.title('BIOMASS_maintenance')
plt.axhline(-np.log10(0.05), color='black', linestyle='--')
plt.axvline(0, color='black', linestyle='--')

def create_legend_entry(label, size, color):
    line = Line2D([0], [0], marker='o', color='w', label=label, markersize=np.sqrt(size), markerfacecolor=color)
    arrow = FancyArrowPatch((0, 0), (1, 0), mutation_scale=10, color=color)
    return line, arrow

# Create legend entries
small_line, small_arrow = create_legend_entry('', 50, 'blue')
medium_line, medium_arrow = create_legend_entry('', 100, 'blue')
large_line, large_arrow = create_legend_entry('', 150, 'blue')
Super_large_line, Super_large_arrow = create_legend_entry('', 200, 'blue')

legend_elements = [small_line, medium_line, large_line, Super_large_line]
plt.legend(handles=legend_elements, title='Effect Size', bbox_to_anchor=(1.2, 1.04), loc='upper right', frameon=False)
ax = plt.gca()

plt.savefig('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_UC/Figure/UC_BIOMASS_maintenance', dpi=900)
plt.show()

# CD study

In [ ]:
# CD healthy
# Run FVA with context-specific reconstructions of healthy samples in the CD study
files_path = os.listdir('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/GSE164985/FAV_per_sample/model_imat/Healthy/')
folder = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/GSE164985/FAV_per_sample/model_imat/Healthy/'
for i in files_path:
    model_path = folder + i
    print(model_path)
    model = cobra.io.load_matlab_model(model_path)
    model.objective = ['ACSm', 'FACOAL40im', 'BIOMASS_maintenance']
    model.solver = 'cplex'
    print(model.objective.expression)
    fva_output = flux_variability_analysis(model)
    save_path = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/GSE164985/FAV_per_sample/FVA_outputs/'
    file_name = i.split('.mat')[0]
    save_path = save_path + file_name + '.csv'
    fva_output.to_csv(save_path)
    print('------------------------------')
        

In [ ]:
# CD Inflammatory
# Run FVA with context-specific reconstructions of inflammatory samples in the CD study
files_path = os.listdir('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/GSE164985/FAV_per_sample/model_imat/Inflamed/')
folder = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/GSE164985/FAV_per_sample/model_imat/Inflamed/'
for i in files_path:
    model_path = folder + i
    print(model_path)
    model = cobra.io.load_matlab_model(model_path)
    model.objective = ['ACSm', 'FACOAL40im', 'BIOMASS_maintenance']
    model.solver = 'cplex'
    print(model.objective.expression)
    fva_output = flux_variability_analysis(model)
    save_path = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/GSE164985/FAV_per_sample/FVA_outputs/'
    file_name = i.split('.mat')[0]
    save_path = save_path + file_name + '.csv'
    fva_output.to_csv(save_path)
    print('------------------------------')

In [ ]:
# Merge maximum flux of the FVA outputs above
reactions = []
CD189 = {}
CD299 = {}
CD364 = {}
HT206 = {}
HT214 = {}
HT216 = {}
HT217 = {}
CD189_fva = pd.read_csv('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/GSE164985/FVA_per_sample/FVA_outputs/CD189.csv')
for i in range(len(CD189_fva)):
    rxn = CD189_fva.iloc[i,0]
    maximum = CD189_fva.iloc[i,2]
    reactions.append(rxn)
    CD189[rxn] = maximum

CD299_fva = pd.read_csv('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/GSE164985/FVA_per_sample/FVA_outputs/CD299.csv')
for i in range(len(CD299_fva)):
    rxn = CD299_fva.iloc[i,0]
    maximum = CD299_fva.iloc[i,2]
    reactions.append(rxn)
    CD299[rxn] = maximum

CD364_fva = pd.read_csv('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/GSE164985/FVA_per_sample/FVA_outputs/CD364.csv')
for i in range(len(CD364_fva)):
    rxn = CD364_fva.iloc[i,0]
    maximum = CD364_fva.iloc[i,2]
    reactions.append(rxn)
    CD364[rxn] = maximum

HT206_fva = pd.read_csv('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/GSE164985/FVA_per_sample/FVA_outputs/HT206.csv')
for i in range(len(HT206_fva)):
    rxn = HT206_fva.iloc[i,0]
    maximum = HT206_fva.iloc[i,2]
    reactions.append(rxn)
    HT206[rxn] = maximum

HT214_fva = pd.read_csv('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/GSE164985/FVA_per_sample/FVA_outputs/HT214.csv')
for i in range(len(HT214_fva)):
    rxn = HT214_fva.iloc[i,0]
    maximum = HT214_fva.iloc[i,2]
    reactions.append(rxn)
    HT214[rxn] = maximum


HT216_fva = pd.read_csv('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/GSE164985/FVA_per_sample/FVA_outputs/HT216.csv')
for i in range(len(HT216_fva)):
    rxn = HT216_fva.iloc[i,0]
    maximum = HT216_fva.iloc[i,2]
    reactions.append(rxn)
    HT216[rxn] = maximum

HT217_fva = pd.read_csv('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/GSE164985/FVA_per_sample/FVA_outputs/HT217.csv')
for i in range(len(HT217_fva)):
    rxn = HT217_fva.iloc[i,0]
    maximum = HT217_fva.iloc[i,2]
    reactions.append(rxn)
    HT217[rxn] = maximum


reaction_deduplicated = list(set(reactions))
len(reaction_deduplicated)

workbook =xlsxwriter.Workbook('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/GSE164985/FVA_per_sample/Results_analysis/FVA_mergedAll.xlsx', {"nan_inf_to_errors": True})
worksheet = workbook.add_worksheet()
worksheet.write(0,1,'CD189')
worksheet.write(0,2,'CD299')
worksheet.write(0,3,'CD364')
worksheet.write(0,4,'HT206')
worksheet.write(0,5,'HT214')
worksheet.write(0,6,'HT216')
worksheet.write(0,7,'HT217')
reaction_CD189 = list(CD189.keys())
reaction_CD299 = list(CD299.keys())
reaction_CD364 = list(CD364.keys())
reaction_HT206 = list(HT206.keys())
reaction_HT214 = list(HT214.keys())
reaction_HT216 = list(HT216.keys())
reaction_HT217 = list(HT217.keys())
for i in range(len(reaction_deduplicated)):
    reaction = reaction_deduplicated[i]
    worksheet.write(i+1, 0, reaction)
    if reaction in reaction_CD189:
        value = CD189[reaction]
        print(value)
        worksheet.write(i+1, 1, value)
    else:
        value = np.nan
        print('NONE')
        worksheet.write(i+1, 1, value)


    if reaction in reaction_CD299:
        value = CD299[reaction]
        print(value)
        worksheet.write(i+1, 2, value)
    else:
        value = np.nan
        print('NONE')
        worksheet.write(i+1, 2, value)


    if reaction in reaction_CD364:
        value = CD364[reaction]
        print(value)
        worksheet.write(i+1, 3, value)
    else:
        value = np.nan
        print('NONE')
        worksheet.write(i+1, 3, value)

    if reaction in reaction_HT206:
        value = HT206[reaction]
        print(value)
        worksheet.write(i+1, 4, value)
    else:
        value = np.nan
        print('NONE')
        worksheet.write(i+1, 4, value)

    if reaction in reaction_HT214:
        value = HT214[reaction]
        print(value)
        worksheet.write(i+1, 5, value)
    else:
        value = np.nan
        print('NONE')
        worksheet.write(i+1, 5, value)

    if reaction in reaction_HT216:
        value = HT216[reaction]
        print(value)
        worksheet.write(i+1, 6, value)
    else:
        value = np.nan
        print('NONE')
        worksheet.write(i+1, 6, value)

    if reaction in reaction_HT217:
        value = HT217[reaction]
        print(value)
        worksheet.write(i+1, 7, value)
    else:
        value = np.nan
        print('NONE')
        worksheet.write(i+1, 7, value)

workbook.close()

In [ ]:
# CD T-test 
CD_data = pd.read_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/GSE164985/FVA_per_sample/Results_analysis/FVA_mergedAll.xlsx', index_col=0)
p_val = []
for i in range(len(CD_data)):
    rxn_id = CD_data.index[i]
    rxn_fluxes = CD_data.loc[rxn_id].tolist()
    ifla_flux = rxn_fluxes[0:3]
    ht_flux  = rxn_fluxes[3:]
    try:
        t_statistic, p_value = ttest_ind(ifla_flux, ht_flux)
        p_val.append(p_value)
    except:
        p_val.append(np.nan)

CD_data['p_value'] = p_val
CD_data.to_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/GSE164985/FVA_per_sample/Results_analysis/CD_t_test.xlsx')

#### Run Flux sampling with all context-specific reconstructions in the CD study

In [ ]:
CD_folder = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/GSE164985/FVA_per_sample/mode_imat/'

for i in ['Healthy', 'Inflamed']:
    folder =  CD_folder + i
    print(folder)
    file_names = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    print(file_names)

    for i in range(len(file_names)):
        print(i)
        filename  = file_names[i]
        print(filename)
        file_path = folder + '/' + filename
        model = load_matlab_model(file_path)
        #model.objective = ['ACSm', 'FACOAL40im', 'BIOMASS_maintenance']

        try:
            achr = ACHRSampler(model, thinning=10)
            s1 = achr.sample(100)
            save_path = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Flux sampling/CD/'
            save_file_name = filename.split('.xml')[0]
            save_file = save_path + save_file_name + '.csv'
            s1.to_csv(save_file)
        except:
            print('cannnot obtain sampling')


In [ ]:
def pad_lists_to_same_length(data, pad_value=np.nan):
    max_length = max(len(lst) for lst in data.values())
    for key, lst in data.items():
        if len(lst) < max_length:
            data[key].extend([pad_value] * (max_length - len(lst)))
    return data

In [ ]:
# combined fluxes from each group of CD
cd_flux_files = os.listdir('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Flux sampling/CD')
cd_flux_path = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Flux sampling/CD/'

# CD_inflamed 
inflam_dict = {}
for i in cd_flux_files:
    if i.startswith('CD'):
        file_path = cd_flux_path + '/' + i
        fluxes_data = pd.read_csv(file_path, index_col=0)
        rxn_list = fluxes_data.columns.to_list()
        for i in rxn_list:
            rxn = i
            rxn_flux = fluxes_data[rxn].to_list()
            if rxn not in inflam_dict.keys():
                inflam_dict[rxn] = rxn_flux
            else:
                ori_flux = inflam_dict[rxn]
                combined_flux = ori_flux + rxn_flux
                inflam_dict[rxn] = combined_flux
        inflam_dict = pad_lists_to_same_length(inflam_dict)
df_inflam = pd.DataFrame(inflam_dict)
df_inflam.to_csv('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Flux sampling/Output_updated/CD_inflam_combined.csv')




health_dict = {}
for i in cd_flux_files:
    if i.startswith('HT'):
        file_path = cd_flux_path + '/' + i
        fluxes_data = pd.read_csv(file_path, index_col=0)
        rxn_list = fluxes_data.columns.to_list()
        for i in rxn_list:
            rxn = i
            rxn_flux = fluxes_data[rxn].to_list()
            if rxn not in health_dict.keys():
                health_dict[rxn] = rxn_flux
            else:
                ori_flux = health_dict[rxn]
                combined_flux = ori_flux + rxn_flux
                health_dict[rxn] = combined_flux
        health_dict = pad_lists_to_same_length(health_dict)
df_HT = pd.DataFrame(health_dict)
df_HT.to_csv('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Flux sampling/Output_updated/CD_HT_combined.csv')


cd_inflammation = pd.read_csv('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Flux sampling/Output/CD_inflam_combined.csv', index_col=0)
cd_health = pd.read_csv('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Flux sampling/Output/CD_HT_combined.csv', index_col=0)

workbook = xlsxwriter.Workbook('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Flux sampling/Output/CD_combinedFlux_comparsion.xlsx')
sheet1 = workbook.add_worksheet()


sheet1.write(0, 0, 'CD_inflammation_vs_health')
sheet1.write(1, 0, 'Reactions')
sheet1.write(1, 1, 'Subsystem')
sheet1.write(1, 2, 'Flux_CD')
sheet1.write(1, 3, 'Flux_HT')
sheet1.write(1, 4, 'P-value')
inflam_rxns = cd_inflammation.columns.to_list()
health_rxns = cd_health.columns.to_list()
overlapped_rxn = list(set(inflam_rxns).intersection(health_rxns))

row_index = 2
for rxn in overlapped_rxn:
    #print(rxn)
    sheet1.write(row_index, 0, rxn)
    
    inflam_fluxes = cd_inflammation[rxn].to_list()
    inflam_fluxes = [x for x in inflam_fluxes if not math.isnan(x)]
    health_fluxes = cd_health[rxn].to_list()
    health_fluxes = [x for x in health_fluxes if not math.isnan(x)]
    inflam_fluxes_mean = mean(inflam_fluxes)
    health_fluxes_mean = mean(health_fluxes)
    sheet1.write(row_index, 2, inflam_fluxes_mean)
    sheet1.write(row_index, 3, health_fluxes_mean)
    t_statistic, p_value = ttest_ind(inflam_fluxes, health_fluxes)
    sheet1.write(row_index, 4, p_value)
    row_index +=1



workbook.close()


In [ ]:
# build combined flux distribution plot of CD
sampling_input = pd.read_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Flux sampling/DistributePlot/Flux sampling inputs.xlsx')
Nuceltide_list = sampling_input['Nucleotide interconversion'].to_list()
fatty_acid_list = sampling_input['Fatty acid metabolism'].to_list()
Purine_list = sampling_input['Purine metabolism'].to_list()


CD_inflam_fluxes = pd.read_csv('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Flux sampling/Output/CD_HT_combined.csv', index_col=0)
CD_health_fluxes = pd.read_csv('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/Flux sampling/Output/CD_inflam_combined.csv', index_col=0)

save_folder_fatty = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Flux sampling/DistributePlot_0525_2024/Fatty acid/'

for i in fatty_acid_list:
    reaction = i
    print(reaction)
    try:
        CD_inflam_data = CD_inflam_fluxes[reaction].to_list()
        CD_inflam_data = [x for x in CD_inflam_data if not math.isnan(x)]
        CD_health_data = CD_health_fluxes[reaction].to_list()
        CD_health_data = [x for x in CD_health_data if not math.isnan(x)]

        # Sample data
        data1 = CD_inflam_data
        data2 = CD_health_data

        # Create distribution plot for data1 and data2
        sns.histplot(data1, kde=True, color='red', label='Inflammatory group', alpha=0.5)
        sns.histplot(data2, kde=True, color='blue', label='Healthy group', alpha=0.5)

        plt.xlabel('Flux value (mmol/gDW/h)')
        plt.ylabel('Density')
        Plot_title = 'CD study: ' + reaction
        plt.title(Plot_title)
        plt.legend()
        save_path = save_folder_fatty + 'CD_' + reaction + '.png'
        plt.savefig(save_path,dpi=600)
        plt.show()
    except:
        continue

#### Single gene knockout simulation in CD

In [ ]:
icolon = read_sbml_model('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Draft_reconstructions/Draft_reconstruction_consensus/icolonoEpithelium.xml')
reactions_all = ['BIOMASS_maintenance', 'ACSm', 'FACOAL40im', 
                 'HACD1m',
                 'ECOAH1m',
                 '5HOXINDACTOX',
                 'ECOAH1x',
                 'HACD1x',
                 '3HKYNAKGAT',
                 '5HOXINOXDA',
                 'RE2349C',
                 'r0647',
                 'ACACT1m']
merged_df = pd.DataFrame({'reaction_id': reactions_all})
merged_df.index = merged_df['reaction_id']

files_path = os.listdir('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/GSE164985/FVA_per_sample/model_imat/Healthy/')
folder = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/GSE164985/FVA_per_sample/model_imat/Healthy/'
print(files_path)

for i in range(len(files_path)):
    print(i)
    file_name = files_path[i]
    print(file_name)
    merged_df = pd.DataFrame({'reaction_id': reactions_all})
    merged_df.index = merged_df['reaction_id']
    
    save_path = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_CD/' + file_name + '_knock.xlsx'

    
    model_path = folder + file_name
    model = load_matlab_model(model_path)
    reaction = [i.id for i in model.reactions]
    reactions_all_filter = list(set(reactions_all)&set(reaction))
    model.solver = 'cplex'
    print(i)
    index = 0
    for i in model.genes:
        
        if index % 100 == 0:
            print(index)
        index +=1 
        gene_id = i.id
        
        model_copy = model.copy()
        model_copy.genes.get_by_id(gene_id).knock_out()
        fva = flux_variability_analysis(model_copy, reactions_all_filter, processes = 6)
        fva = fva.drop(columns=['minimum'])
        fva.rename(columns={'maximum': gene_id}, inplace=True)
        merged_df = pd.merge(merged_df,fva, left_index=True, right_index=True, how='left')
        
    merged_df.to_excel(save_path)
    print('complete knockout:', file_name)




colon = read_sbml_model('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Draft_reconstructions/Draft_reconstruction_consensus/icolonoEpithelium.xml')
reactions_all = ['BIOMASS_maintenance', 'ACSm', 'FACOAL40im', 
                 'HACD1m',
                 'ECOAH1m',
                 '5HOXINDACTOX',
                 'ECOAH1x',
                 'HACD1x',
                 '3HKYNAKGAT',
                 '5HOXINOXDA',
                 'RE2349C',
                 'r0647',
                 'ACACT1m']
merged_df = pd.DataFrame({'reaction_id': reactions_all})
merged_df.index = merged_df['reaction_id']

files_path = os.listdir('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/GSE164985/FVA_per_sample/model_imat/Inflamed/')
folder = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/GSE164985/FVA_per_sample/model_imat/Inflamed/'
print(files_path)

for i in range(len(files_path)):
    print(i)
    file_name = files_path[i]
    print(file_name)
    merged_df = pd.DataFrame({'reaction_id': reactions_all})
    merged_df.index = merged_df['reaction_id']
    
    save_path = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_CD/' + file_name + '_knock.xlsx'

    
    model_path = folder + file_name
    model = load_matlab_model(model_path)
    reaction = [i.id for i in model.reactions]
    reactions_all_filter = list(set(reactions_all)&set(reaction))
    model.solver = 'cplex'
    print(i)
    index = 0
    for i in model.genes:
        
        if index % 100 == 0:
            print(index)
        index +=1 
        gene_id = i.id
        
        model_copy = model.copy()
        model_copy.genes.get_by_id(gene_id).knock_out()
        fva = flux_variability_analysis(model_copy, reactions_all_filter, processes = 9)
        fva = fva.drop(columns=['minimum'])
        fva.rename(columns={'maximum': gene_id}, inplace=True)
        merged_df = pd.merge(merged_df,fva, left_index=True, right_index=True, how='left')
        
    merged_df.to_excel(save_path)
    print("knock completed:", file_name)


In [ ]:
# process data
reactions_all = [i.id for i in icolon.genes]
merged_df = pd.DataFrame({'': reactions_all})
merged_df.index = merged_df['']
                             
merged_biomass = merged_df['']
merged_acsm = merged_df['']
merge_FACOAL40im = merged_df['']
merge_ACACT1m = merged_df['']
merge_ECOAH1m = merged_df['']
merge_5HOXINOXDA = merged_df['']
merge_5HOXINDACTOX = merged_df['']
merge_3HKYNAKGAT = merged_df['']


files = os.listdir('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_CD')
folder = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_CD/'
files.remove('processed')
files.remove('plot_dada')


files.sort()

for i in files:
    print(i)
    sample_id = i.split('.mat_knock.xlsx')[0]
    print(sample_id)
    file_path = folder + i
    df_ko = pd.read_excel(file_path)
    df_ko.index = df_ko['reaction_id']

    selected_rxn = ['BIOMASS_maintenance', 
                    'ACSm', 
                    'FACOAL40im', 
                    'ACACT1m', 
                    'ECOAH1m', 
                    '5HOXINOXDA', 
                    '5HOXINDACTOX', 
                    '3HKYNAKGAT']

    df_ko = df_ko.loc[selected_rxn]
    

    col1 = sample_id + '_BIOMASS_maintenance'
    col2 = sample_id + '_ACSm'
    col3 = sample_id + '_FACOAL40im'
    col4 = sample_id + '_ACACT1m'
    col5 = sample_id + '_ECOAH1m'
    col6 = sample_id + '_5HOXINOXDA'
    col7 = sample_id + '_5HOXINDACTOX'
    col8 = sample_id + '_3HKYNAKGAT'

    df_ko_t = df_ko.T
    df_ko_t.columns =[col1, col2, col3, col4, col5, col6, col7, col8]
    df_ko_t = df_ko_t.iloc[2:]


    wt_df_folder = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Case_Studies/GSE164985/FVA_per_sample/FAV_outputs/'
    wt_df_folder_path = wt_df_folder  + sample_id + '.csv'
    wt_df = pd.read_csv(wt_df_folder_path, index_col=0)
   
    wt_BIOMASS = wt_df.loc['BIOMASS_maintenance', 'maximum']
    if wt_BIOMASS == 0:
        wt_BIOMASS = 1e-8
        print('set wt_BIOMASS as 1e-8')
    else:
        print(wt_BIOMASS)

    wt_acsm = wt_df.loc['ACSm', 'maximum']
    if wt_acsm == 0:
        wt_acsm = 1e-8
        print('set wt_acsm as 1e-8')
    else:
        print(wt_acsm)
   
    wt_FACOAL40im = wt_df.loc['FACOAL40im', 'maximum']
    if wt_FACOAL40im == 0:
        wt_FACOAL40im = 1e-8
        print('set wt_FACOAL40im as 1e-8')
    else:
        print(wt_FACOAL40im)

    try:
        wt_ACACT1m = wt_df.loc['ACACT1m', 'maximum']
        if wt_ACACT1m == 0:
            wt_ACACT1m = 1e-8
            print('set wt_ACACT1m as 1e-8')
        else:
            print(wt_ACACT1m)
    except:
        wt_ACACT1m = 1e-8
        print('set wt_ACACT1m as 1e-8')

    try:
        wt_ECOAH1m = wt_df.loc['ECOAH1m', 'maximum']
        if wt_ECOAH1m == 0:
            wt_ECOAH1m = 1e-8
            print('set wt_ECOAH1m as 1e-8')
        else:
            print(wt_ECOAH1m)
    except:
        wt_ECOAH1m = 1e-8
        print('set wt_ECOAH1m as 1e-8')


    try:
        wt_5HOXINDACTOX = wt_df.loc['5HOXINDACTOX', 'maximum']
        if wt_5HOXINDACTOX == 0:
            wt_5HOXINDACTOX = 1e-8
            print('set wt_5HOXINDACTOX as 1e-8')
        else:
            print(wt_5HOXINDACTOX)
    except:
        wt_5HOXINDACTOX = 1e-8
        print('set wt_5HOXINDACTOX as 1e-8')

    
    try:
        wt_5HOXINOXDA = wt_df.loc['5HOXINOXDA', 'maximum']
        if wt_5HOXINOXDA == 0:
            wt_5HOXINOXDA = 1e-8
            print('set wt_5HOXINOXDA as 1e-8')
        else:
            print(wt_5HOXINOXDA)
    except:
        wt_5HOXINOXDA = 1e-8
        print('set wt_5HOXINOXDA as 1e-8')


    try:
        wt_3HKYNAKGAT = wt_df.loc['3HKYNAKGAT', 'maximum']
        if wt_3HKYNAKGAT == 0:
            wt_3HKYNAKGAT = 1e-8
            print('set wt_3HKYNAKGAT as 1e-8')
        else:
            print(wt_3HKYNAKGAT)
    except:
        wt_3HKYNAKGAT = 1e-8
        print('set wt_3HKYNAKGAT as 1e-8')


    

    BIOMASS = df_ko_t.iloc[:, 0:1]
    BIOMASS = (wt_BIOMASS-BIOMASS)/wt_BIOMASS
    merged_biomass = pd.merge(merged_biomass, BIOMASS, left_index=True, right_index=True, how='left')

    acsm = df_ko_t.iloc[:, 1:2]
    acsm = (wt_acsm-acsm)/wt_acsm
    merged_acsm = pd.merge(merged_acsm, acsm, left_index=True, right_index=True, how='left')

    FACOAL40im = df_ko_t.iloc[:, 2:3]
    FACOAL40im = (wt_FACOAL40im-FACOAL40im)/wt_FACOAL40im
    merge_FACOAL40im = pd.merge(merge_FACOAL40im, FACOAL40im, left_index=True, right_index=True, how='left')

    ACACT1m = df_ko_t.iloc[:, 3:4]
    ACACT1m = (wt_ACACT1m-ACACT1m)/wt_ACACT1m
    merge_ACACT1m = pd.merge(merge_ACACT1m, ACACT1m, left_index=True, right_index=True, how='left')

    ECOAH1m = df_ko_t.iloc[:, 4:5]
    ECOAH1m = (wt_ECOAH1m-ECOAH1m)/wt_ECOAH1m
    merge_ECOAH1m = pd.merge(merge_ECOAH1m, ECOAH1m, left_index=True, right_index=True, how='left')

    _5HOXINOXDA = df_ko_t.iloc[:, 5:6]
    _5HOXINOXDA = (wt_5HOXINOXDA-_5HOXINOXDA)/wt_5HOXINOXDA
    merge_5HOXINOXDA = pd.merge(merge_5HOXINOXDA, _5HOXINOXDA, left_index=True, right_index=True, how='left')

    _5HOXINDACTOX = df_ko_t.iloc[:, 6:7]
    _5HOXINDACTOX = (wt_5HOXINDACTOX-_5HOXINDACTOX)/wt_5HOXINDACTOX
    merge_5HOXINDACTOX = pd.merge(merge_5HOXINDACTOX, _5HOXINDACTOX, left_index=True, right_index=True, how='left')

    _3HKYNAKGAT = df_ko_t.iloc[:, 7:8]
    _3HKYNAKGAT = (wt_3HKYNAKGAT-_3HKYNAKGAT)/wt_3HKYNAKGAT
    merge_3HKYNAKGAT = pd.merge(merge_3HKYNAKGAT, _3HKYNAKGAT, left_index=True, right_index=True, how='left')

    

merged_biomass = merged_biomass.drop(merged_biomass.columns[0], axis=1)
merged_acsm = merged_acsm.drop(merged_acsm.columns[0], axis=1)
merge_FACOAL40im = merge_FACOAL40im.drop(merge_FACOAL40im.columns[0], axis=1)
merge_ACACT1m = merge_ACACT1m.drop(merge_ACACT1m.columns[0], axis=1)
merge_ECOAH1m = merge_ECOAH1m.drop(merge_ECOAH1m.columns[0], axis=1)
merge_5HOXINOXDA = merge_5HOXINOXDA.drop(merge_5HOXINOXDA.columns[0], axis=1)
merge_5HOXINDACTOX = merge_5HOXINDACTOX.drop(merge_5HOXINDACTOX.columns[0], axis=1)
merge_3HKYNAKGAT = merge_3HKYNAKGAT.drop(merge_3HKYNAKGAT.columns[0], axis=1)   
    


# change gene names
new_row_names  = {}
for i in df_ko_t.index:
    new_row_names[i] = icolon.genes.get_by_id(i).name

merged_biomass = merged_biomass.rename(index=new_row_names)
merged_biomass.to_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_CD/processed/CD_BIOMASS.xlsx')

merged_acsm = merged_acsm.rename(index=new_row_names)
merged_acsm.to_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_CD/processed/CD_ACSm.xlsx')

merge_FACOAL40im = merge_FACOAL40im.rename(index=new_row_names)
merge_FACOAL40im.to_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_CD/processed/CD_FACOAL40im.xlsx')

merge_ACACT1m = merge_ACACT1m.rename(index=new_row_names)
merge_ACACT1m.to_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_CD/processed/CD_ACACT1m.xlsx')

merge_ECOAH1m = merge_ECOAH1m.rename(index=new_row_names)
merge_ECOAH1m.to_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_CD/processed/CD_ECOAH1m.xlsx')

merge_5HOXINOXDA = merge_5HOXINOXDA.rename(index=new_row_names)
merge_5HOXINOXDA.to_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_CD/processed/CD_5HOXINOXDA.xlsx')

merge_5HOXINDACTOX = merge_5HOXINDACTOX.rename(index=new_row_names)
merge_5HOXINDACTOX.to_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_CD/processed/CD_5HOXINDACTOX.xlsx')

merge_3HKYNAKGAT = merge_3HKYNAKGAT.rename(index=new_row_names)
merge_3HKYNAKGAT.to_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_CD/processed/CD_3HKYNAKGAT.xlsx')


In [ ]:
folder = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_CD/processed/'
files = os.listdir(folder)

for i in files:
    file_path = folder + i
    print(i)
    df = pd.read_excel(file_path, index_col=0)
    df.sort_values(by=df.columns.tolist(), ascending=True, inplace=True)
    cd_mean  = df.iloc[:, :3].mean(axis=1)
    ht_mean  = df.iloc[:, 3:7].mean(axis=1)
    all_mean = df.iloc[:, 0:7].mean(axis=1)
    ratio = abs(cd_mean/ht_mean)
    try:
        ratio_log2 = ratio.apply(lambda x: math.log2(x))
    except:
        cd_mean  = df.iloc[:, :3].mean(axis=1)
        cd_mean = cd_mean.replace(0, 1e-8)
        ht_mean  = df.iloc[:, 3:7].mean(axis=1)
        all_mean = df.iloc[:, 0:7].mean(axis=1)
        ratio = abs(cd_mean/ht_mean)
        ratio_log2 = ratio.apply(lambda x: math.log2(x))

    ratio_log2 = ratio_log2.to_frame()
    
    # t-test
    cd = df.iloc[:, :3]
    ht = df.iloc[:, 3:7]
    t_statistic, p_values = ttest_ind(cd, ht, axis=1, nan_policy='omit')

    geneID = cd.index
    geneID = geneID.to_list()
    p_values = list(p_values)
    ttest_results = pd.DataFrame({'GeneID':geneID, 'p-valve': p_values})
    ttest_results.index = ttest_results['GeneID']
    ttest_results = ttest_results.drop('GeneID', axis=1)
    
    ratio_log2.rename(columns={ratio_log2.columns[0]: 'log2(CD/WT)'}, inplace=True)
    ratio_log2['all mean'] = all_mean
    volcano_data = pd.merge(ratio_log2, ttest_results, left_index=True, right_index=True, how='left')
    volcanoPlot_data_save = '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_CD/plot_dada/' + i
    print(volcanoPlot_data_save)
    volcano_data.to_excel(volcanoPlot_data_save)



In [ ]:
# Build  volcano plot of CD study
volcano_data = pd.read_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_CD/plot_dada/CD_BIOMASS.xlsx', index_col=0)

log_fold_change = volcano_data['log2(CD/WT)']
p_values = volcano_data['p-valve']
p_values = -np.log10(p_values)
all_mean = volcano_data['all mean']
gene_id = volcano_data.index.to_list()
threshold = -np.log10(0.05)

sizes = np.linspace(1, 250, len(all_mean))
colors = ['red' if fc > 0 else 'blue' for fc in log_fold_change]

plt.figure(figsize=(8, 6))
plt.scatter(log_fold_change, p_values, s=sizes, c=colors, alpha=0.5)

text = []
font_props = {'family': 'monospace', 'size': 8, 'weight': 'bold'}
for x, y, l in zip(log_fold_change, p_values, gene_id):
    if y > -np.log10(0.05):
        text.append(plt.text(x, y, l, size=10, ha='center', va='center', fontdict=font_props))

adjust_text(text, expand=(5, 7),arrowprops=dict(arrowstyle="->", color='k', lw=1))       
plt.xlabel('log2(Inflammatory group/Healthy group)')
plt.ylabel('-log10(p-value)')
plt.title('BIOMASS_maintenance')
plt.axhline(-np.log10(0.05), color='black', linestyle='--')
plt.axvline(0, color='black', linestyle='--')

def create_legend_entry(label, size, color):
    line = Line2D([0], [0], marker='o', color='w', label=label, markersize=np.sqrt(size), markerfacecolor=color)
    arrow = FancyArrowPatch((0, 0), (1, 0), mutation_scale=10, color=color)
    return line, arrow

# Create legend entries
small_line, small_arrow = create_legend_entry('', 50, 'blue')
medium_line, medium_arrow = create_legend_entry('', 100, 'blue')
large_line, large_arrow = create_legend_entry('', 150, 'blue')
Super_large_line, Super_large_arrow = create_legend_entry('', 200, 'blue')

legend_elements = [small_line, medium_line, large_line, Super_large_line]
plt.legend(handles=legend_elements, title='Effect Size', bbox_to_anchor=(1.2, 1.04), loc='upper right', frameon=False)



ax = plt.gca()
plt.savefig('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Manuscript/Knockout/Knockout_simulation_CD/Figure/CD_BIOMASS_maintenance', dpi=900)

plt.show()